## Stream Processing Exercise 4 - Consuming from Kafka

Goals:

* Perform different computations on a input stream: read, aggregation, windowed aggregation
* Additional references
    * [Spark Streaming](https://spark.apache.org/streaming/)
    * [Structured Spark Streaming documentation](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
    * [Spark and Kafka integration guide](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html)


Let’s inspect content of Pageviews topic, showing it every 5 seconds:

In [1]:
import sys
import os 
import json
import hashlib
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.streaming import StreamingContext

from pyspark.streaming.kafka import KafkaUtils
import json
#from org.elasticsearch.spark import *

conf = SparkConf().setAll([("es.index.auto.create", "true")])

sc = SparkContext(appName="QuotesConsumer",conf=conf)

#sc = SparkContext(appName="PageViewsConsumer")

ssc = StreamingContext(sc, 5)


topicQuotes = ['quotes']

kafkaParams = {'bootstrap.servers': 'broker:29092', 
               'group.id' : 'quotesConsumer'}

streamQuotes = KafkaUtils.createDirectStream(ssc, topicQuotes, kafkaParams)

streamQuotes.map(lambda record : (record[0], record[1])).pprint()


es_write_quotes = {
        "es.nodes" : "elasticsearch",
        "es.port" : "9200",
        "es.resource" : 'quotes/_doc',
        "es.input.json": "yes",
    }

streamQuotes.foreachRDD(lambda rdd: rdd.saveAsNewAPIHadoopFile(
        path='-',
        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
        keyClass="org.apache.hadoop.io.NullWritable",
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
        conf=es_write_quotes))

topicTwitter = ['tweets']

streamTwitter = KafkaUtils.createDirectStream(ssc, topicTwitter, kafkaParams)

streamTwitter.map(lambda record : (record[0], record[1])).pprint()


es_write_twitter = {
        "es.nodes" : "elasticsearch",
        "es.port" : "9200",
        "es.resource" : 'tweets/_doc',
        "es.input.json": "yes",
    }

streamTwitter.foreachRDD(lambda rdd: rdd.saveAsNewAPIHadoopFile(
        path='-',
        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
        keyClass="org.apache.hadoop.io.NullWritable",
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
        conf=es_write_twitter))

ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2021-01-14 17:17:25
-------------------------------------------

-------------------------------------------
Time: 2021-01-14 17:17:25
-------------------------------------------
(None, '{"created_at":"Thu Jan 14 17:17:18 +0000 2021","id":1349767560363868163,"id_str":"1349767560363868163","text":"@Phaethon314 The Ivy Leaguers are overrated. I worked with many at Google. Arrogant Brotopia.","display_text_range":[13,93],"source":"\\u003ca href=\\"https:\\/\\/mobile.twitter.com\\" rel=\\"nofollow\\"\\u003eTwitter Web App\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":1349767185405792257,"in_reply_to_status_id_str":"1349767185405792257","in_reply_to_user_id":1289187183216263168,"in_reply_to_user_id_str":"1289187183216263168","in_reply_to_screen_name":"Phaethon314","user":{"id":1349331096471703555,"id_str":"1349331096471703555","name":"Sabrinagals","screen_name":"sabrinagals","location":null,"url":null,"description":"Big\\u2764

Py4JJavaError: An error occurred while calling o29.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/local/spark/python/pyspark/streaming/kafka.py", line 403, in <lambda>
    func = lambda r, rdd: old_func(rdd)
  File "<ipython-input-1-6a699efa4abb>", line 65, in <lambda>
    conf=es_write_twitter))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1438, in saveAsNewAPIHadoopFile
    keyConverter, valueConverter, jconf)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling z:org.apache